In [50]:
import io
import os
import re
import shutil
import string
import tensorflow as tf
import numpy as np

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM
from tensorflow.keras.layers import TextVectorization

In [4]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1.tar.gz", url, untar = True, cache_dir = '.', cache_subdir = '')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
os.listdir(dataset_dir)

84140032/84125825 [==============================] - 51s 1us/step


['imdbEr.txt', 'test', 'imdb.vocab', 'README', 'train']

In [5]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

['urls_unsup.txt',
 'neg',
 'urls_pos.txt',
 'unsup',
 'urls_neg.txt',
 'pos',
 'unsupBow.feat',
 'labeledBow.feat']

In [14]:
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

In [15]:
batch_size = 1024
seed = 123
train_ds = tf.keras.preprocessing.text_dataset_from_directory('aclImdb/train', batch_size = batch_size, validation_split = 0.2, subset = 'training', seed = seed)
val_ds = tf.keras.preprocessing.text_dataset_from_directory('aclImdb/train', batch_size = batch_size, validation_split = 0.2, subset = 'validation', seed = seed)


Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [18]:
for text_batch, label_batch in train_ds.take(1):
    for i in range(5):
        print(label_batch[i].numpy(), text_batch.numpy()[i])

1 b"The original animated Dark Knight returns in this ace adventure movie that rivals Mask of Phantasm in its coolness. There's a lot of style and intelligence in Mystery of the Batwoman, so much more than Batman Forever or Batman and Robin.<br /><br />There's a new crime-fighter on the streets of Gotham. She dresses like a bat but she's not a grown-up Batgirl. And Batman is denying any affiliation with her. Meanwhile Bruce Wayne has to deal with the usual romances and detective work. But the Penguin, Bain and the local Mob makes things little more complicated.<br /><br />I didn't have high hopes for this 'un since being strongly let down but the weak Batman: Sub Zero (Robin isn't featured so much here!)but I was delighted with the imaginative and exciting set pieces, the clever plot and a cheeky sense of humor. This is definitely a movie no fan of Batman should be without. Keep your ears open for a really catchy song called 'Betcha Neva' which is featured prominently through-out.<br /

In [19]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size = AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size = AUTOTUNE)

In [23]:
#Strip unnecessary HTML tags
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    return tf.strings.regex_replace(stripped_html, '[%s]' % re.escape(string.punctuation), '')

vocab_size = 1000
sequence_length = 100

#Text Vectorization layer 
vectorize_layer = TextVectorization(
    standardize = custom_standardization,
    max_tokens = vocab_size,
    output_mode = 'int',
    output_sequence_length = sequence_length)

#making text-only dataset and adapting to build the vocab in the layer
text_ds = train_ds.map(lambda x,y : x)
vectorize_layer.adapt(text_ds)

In [44]:
embedding_dim = 64

model = Sequential([
    vectorize_layer,
    Embedding(vocab_size, embedding_dim, name = "embedding", mask_zero = True),
    Bidirectional(LSTM(64)),
    Dense(64, activation = 'relu'),
    Dense(1)])

In [45]:
#Tensor Board
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = "logs")

In [46]:
model.compile(optimizer = 'adam',
              loss = tf.keras.losses.BinaryCrossentropy(from_logits = True),
              metrics = ['accuracy'])
model.fit(
    train_ds,
    validation_data = val_ds,
    epochs = 10, 
    callbacks = [tensorboard_callback])

Epoch 1/10
20/20 [==============================] - 24s 975ms/step - loss: 0.6880 - accuracy: 0.5034 - val_loss: 0.6605 - val_accuracy: 0.5474
Epoch 2/10
20/20 [==============================] - 17s 865ms/step - loss: 0.5921 - accuracy: 0.6586 - val_loss: 0.5384 - val_accuracy: 0.7318
Epoch 3/10
20/20 [==============================] - 17s 868ms/step - loss: 0.4707 - accuracy: 0.7690 - val_loss: 0.4481 - val_accuracy: 0.7864
Epoch 4/10
20/20 [==============================] - 17s 871ms/step - loss: 0.4226 - accuracy: 0.7990 - val_loss: 0.4414 - val_accuracy: 0.7866
Epoch 5/10
20/20 [==============================] - 17s 874ms/step - loss: 0.4050 - accuracy: 0.8069 - val_loss: 0.4378 - val_accuracy: 0.7816
Epoch 6/10
20/20 [==============================] - 17s 873ms/step - loss: 0.3992 - accuracy: 0.8124 - val_loss: 0.4364 - val_accuracy: 0.7804
Epoch 7/10
20/20 [==============================] - 17s 878ms/step - loss: 0.3957 - accuracy: 0.8143 - val_loss: 0.4381 - val_accuracy: 0.7804

In [40]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 100)               0         
_________________________________________________________________
embedding (Embedding)        (None, 100, 64)           64000     
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               66048     
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 138,369
Trainable params: 138,369
Non-trainable params: 0
_________________________________________________________________


In [47]:
%load_ext tensorboard
%tensorboard --logdir logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 53577), started 0:49:25 ago. (Use '!kill 53577' to kill it.)

In [48]:
test_loss, test_acc = model.evaluate(val_ds)

print('Test Loss:', test_loss)
print('Test Acc:', test_acc)

5/5 [==============================] - 1s 275ms/step - loss: 0.4679 - accuracy: 0.7824
Test Loss: 0.4678657650947571
Test Acc: 0.7824000120162964


In [79]:

predictions = model.predict(np.array(['The movie was awful','The movie was very good', 'movie was bad']))
#negative value for negative review, positive value for positive review
def val_to_sentiment(val):
    return 'Positive' if val >= 0 else 'Negative'
print([val_to_sentiment(val) for val in predictions])

['Negative', 'Positive', 'Negative']
